<a href="https://colab.research.google.com/github/xechehot/sirius-2021-warehouse/blob/main/Maksim/Correct_create_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/xechehot/sirius-2021-warehouse

Cloning into 'sirius-2021-warehouse'...
remote: Enumerating objects: 419, done.
remote: Counting objects: 100% (419/419), done.
remote: Compressing objects: 100% (313/313), done.
remote: Total 419 (delta 260), reused 225 (delta 95), pack-reused 0
Receiving objects: 100% (419/419), 414.52 KiB | 3.80 MiB/s, done.
Resolving deltas: 100% (260/260), done.


In [2]:
! cd sirius-2021-warehouse; git pull

Already up to date.


In [3]:
from json import load as JLoad

with open('sirius-2021-warehouse/data_sample/sample_64_1.json', 'r') as f:
  data = JLoad(f) 

#data

In [4]:
floors = data['warehouse']['meta']['floors']
block_x = data['warehouse']['meta']['block_x']
block_y = data['warehouse']['meta']['block_y']
rows = data['warehouse']['meta']['rows'] 
sections = data['warehouse']['meta']['sections']

floors, block_x, block_y, rows, sections

(1, 1, 2, 4, 8)

Также нужно добавть информацию о товарах, которые лежат на складе в поле items:

In [11]:
data['warehouse']['stock'][0]

{'count': 4,
 'id': 5277987,
 'p': {'block_x': 1, 'block_y': 2, 'floor': 1, 'row': 3, 'section': 6}}

In [12]:
def get_out_coordinates(stock_item):
    # ТУТ НУЖНО ПРАВИЛЬНО ПОСЧИТАТЬ КООРДИНТЫ x и y из stock_item['p']
    return {
      "position": {
        "x": 0,
        "y": 0
      },
      "id": stock_item['id'],
      "amount": stock_item['count']
    }

In [13]:
output_items = []
for stock_item in data['warehouse']['stock']:
    out_item = get_out_coordinates(stock_item)
    output_items.append(out_item)

In [16]:
import json

Map_y = (3 * rows) // 2 + 1
Map_x = sections + 2
 
s_map_x = block_x * Map_x
s_map_y = block_y * Map_y

map_for_block = [[0 for i in range(s_map_y)] for j in range(s_map_x)]


MAP = []

for i in range(Map_x):
  MAP.append([0 for _ in range(Map_y)])
  if i == 0 or i == Map_x - 1:
    continue
  for j in range(Map_y):
    if j % 3 != 0:
      MAP[i][j] = 1


# print(Map_x, Map_y)



# for i in MAP:
#  for j in i:
  #  print(j, end = ' ')
#  print('')


for i in range(s_map_x):
  for j in range(s_map_y):
    map_for_block[i][j] = MAP[i % Map_x][j % Map_y]

for i in range (s_map_y):
  map_for_block[0][i] = 2

startx = s_map_x - 1
starty = s_map_y // 2

# for i in map_for_block:
#  for j in i:
  #  print(j, end = ' ')
#  print('')

output = {
      "map": {
          "dimensions": {
              "x": s_map_x,
              "y": s_map_y
          },
          "shelves": [
                      
          ],
          "dropZone": [
                  
          ],
          "start": {
            "x": s_map_x - 1,
            "y": s_map_y // 2 
        }
      },
    #   Добавляем items
      "items": output_items,
}

shelves = []
dropZone = []
for i in range(s_map_x):
    for j in range(s_map_y):
        if map_for_block[i][j] == 1:
            item = {
                "x": i,
                "y": j
            }
            shelves.append(item)
        elif map_for_block[i][j] == 2:
            item = {
                "x": i,
                "y": j
            }
            dropZone.append(item)
output['map']['shelves'] = shelves
output['map']['dropZone'] = dropZone
with open("visual_sample_map.json", "w") as f:
  json.dump(output, f, indent=2)
# print(json.dumps(output, indent=3))
